KELAS SAINS DATA SD-04-01
NAMA: JURY MAXWELL
NIM: 103132400023
NAMA: Enggar Tri Rahmadhani
NIM: 103132400007


# Import Libraray

In [4]:
import pandas as pd

# Import Data

In [5]:
# Membaca dataset dari file CSV
file_path = r"C:\Users\jmjur\Documents\IPSD\Fuzzy AI\Fuzzy-Mamdani-Method\Dataset Bengkel AI.csv"
df = pd.read_csv(file_path, sep=";")

df.head()

,id_bengkel,Kualitas_Servis,Harga
0,AUTO01,1,8
1,AUTO02,7,4
2,AUTO03,5,10
3,AUTO04,6,4
4,AUTO05,1,8


# Fungsi Keanggotaan 

In [6]:
# Definisi fungsi keanggotaan segitiga (0-1)
def keanggotaan(x, a, b, c):
    if a == b and x <= b:
        return 1
    if b == c and x >= b:
        return 1

    if x <= a or x >= c:
        return 0
    if x == b:
        return 1
    if x < b:
        return (x - a) / (b - a)
    return (c - x) / (c - b)


# Fuzzy Set

In [7]:
# Definisi himpunan fuzzy untuk servis
set_servis = {
    "buruk" : (1, 1, 5),
    "cukup" : (3, 5, 7),
    "baik"  : (5, 10, 10),
}
# Definisi himpunan fuzzy untuk harga
set_harga = {
    "murah"  : (1, 1, 5),
    "sedang" : (3, 5, 7),
    "mahal"  : (5, 10, 10),
}

# Fuzzification

In [8]:
# Fuzzifikasi nilai input menggunakan himpunan fuzzy
def fuzzifikasi_nilai(x, set_fuzzy):
    hasil = {}
    for label, (a, b, c) in set_fuzzy.items():
        hasil[label] = keanggotaan(x, a, b, c)
    return hasil

df["mu_servis"] = df["Kualitas_Servis"].apply(lambda x: fuzzifikasi_nilai(float(x), set_servis))
df["mu_harga"]  = df["Harga"].apply(lambda x: fuzzifikasi_nilai(float(x), set_harga))

df[["id_bengkel", "Kualitas_Servis", "Harga", "mu_servis", "mu_harga"]].head()


,id_bengkel,Kualitas_Servis,Harga,mu_servis,mu_harga
0,AUTO01,1,8,"{'buruk': 1, 'cukup': 0, 'baik': 0}","{'murah': 0, 'sedang': 0, 'mahal': 0.6}"
1,AUTO02,7,4,"{'buruk': 0, 'cukup': 0, 'baik': 0.4}","{'murah': 0.25, 'sedang': 0.5, 'mahal': 0}"
2,AUTO03,5,10,"{'buruk': 0, 'cukup': 1, 'baik': 0}","{'murah': 0, 'sedang': 0, 'mahal': 1}"
3,AUTO04,6,4,"{'buruk': 0, 'cukup': 0.5, 'baik': 0.2}","{'murah': 0.25, 'sedang': 0.5, 'mahal': 0}"
4,AUTO05,1,8,"{'buruk': 1, 'cukup': 0, 'baik': 0}","{'murah': 0, 'sedang': 0, 'mahal': 0.6}"


# Rule Base

In [9]:
# Definisi aturan fuzzy
rules = [
    ("buruk", "murah",  "sedang"),
    ("buruk", "sedang", "rendah"),
    ("buruk", "mahal",  "rendah"),

    ("cukup", "murah",  "tinggi"),
    ("cukup", "sedang", "sedang"),
    ("cukup", "mahal",  "rendah"),

    ("baik",  "murah",  "tinggi"),
    ("baik",  "sedang", "tinggi"),
    ("baik",  "mahal",  "sedang"),
]

# Inferensi Mamdani

In [10]:
# Definisi himpunan output fuzzy
def inferensi_mamdani(mu_servis, mu_harga):
    output = {"rendah": 0, "sedang": 0, "tinggi": 0}

    for s_label, h_label, y_label in rules:
        alpha = min(mu_servis[s_label], mu_harga[h_label])
        if alpha > output[y_label]:
            output[y_label] = alpha

    return output

df["inferensi_kelayakan_bengkel"] = df.apply(lambda row: inferensi_mamdani(row["mu_servis"], row["mu_harga"]), axis=1)

df[["id_bengkel", "inferensi_kelayakan_bengkel"]].head()

,id_bengkel,inferensi_kelayakan_bengkel
0,AUTO01,"{'rendah': 0.6, 'sedang': 0, 'tinggi': 0}"
1,AUTO02,"{'rendah': 0, 'sedang': 0, 'tinggi': 0.4}"
2,AUTO03,"{'rendah': 1, 'sedang': 0, 'tinggi': 0}"
3,AUTO04,"{'rendah': 0, 'sedang': 0.5, 'tinggi': 0.25}"
4,AUTO05,"{'rendah': 0.6, 'sedang': 0, 'tinggi': 0}"


# Defuzzifikasi

In [18]:
# Definisi himpunan output untuk defuzzifikasi
set_output = {
    "rendah": (0, 0, 50),
    "sedang": (30, 50, 70),
    "tinggi": (50, 100, 100)
}

# Defuzzifikasi menggunakan metode centroid
def defuzzifikasi(out_strength):
    pembilang = 0
    penyebut = 0

    for z in range(0, 101):
        mu_z = 0
        for label, (a, b, c) in set_output.items():
            mu_label = keanggotaan(z, a, b, c)
            mu_z = max(mu_z, min(out_strength[label], mu_label))

        pembilang += z * mu_z
        penyebut += mu_z

    if penyebut == 0:
        return 0
    return round(pembilang / penyebut, 2)

df["Skor_Kelayakan"] = df["inferensi_kelayakan_bengkel"].apply(defuzzifikasi)

df[["id_bengkel", "Kualitas_Servis", "Harga", "Skor_Kelayakan"]].head()


,id_bengkel,Kualitas_Servis,Harga,Skor_Kelayakan
0,AUTO01,1,8,18.31
1,AUTO02,7,4,79.84
2,AUTO03,5,10,16.33
3,AUTO04,6,4,62.05
4,AUTO05,1,8,18.31


# Mendapatkan TOP 5 bengkel

In [19]:
# Mendapatkan 5 bengkel teratas berdasarkan skor kelayakan
top5 = df.sort_values(by="Skor_Kelayakan", ascending=False).head(5)

top5[["id_bengkel", "Kualitas_Servis", "Harga", "Skor_Kelayakan"]]

,id_bengkel,Kualitas_Servis,Harga,Skor_Kelayakan
32,AUTO33,10,5,83.67
94,AUTO95,5,1,83.67
62,AUTO63,10,5,83.67
61,AUTO62,10,1,83.67
12,AUTO13,5,2,82.78


In [20]:
# Menyimpan hasil 5 bengkel teratas ke file CSV
top5.to_csv("Output_Top5_Bengkel.csv", index=False, sep=";")